<a href="https://colab.research.google.com/github/OcamaAslam/Automated-Product-Screenshots/blob/main/fyp1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Dec 28 03:15:43 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [29]:
import nltk
import pandas as pd
from collections import defaultdict, Counter
from nltk.corpus import reuters
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

nltk.download('reuters')
nltk.download('punkt')
nltk.download('stopwords')
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
category_counts = Counter(category for file_id in reuters.fileids() for category in reuters.categories(file_id))
top_categories = [category for category, count in category_counts.most_common(10)]

In [31]:
top_categories

['earn',
 'acq',
 'money-fx',
 'grain',
 'crude',
 'trade',
 'interest',
 'ship',
 'wheat',
 'corn']

In [32]:
feature_category_matrix = defaultdict(lambda: defaultdict(int))

In [33]:
for file_id in reuters.fileids():
    categories = [category for category in reuters.categories(file_id) if category in top_categories]
    words = word_tokenize(reuters.raw(file_id))
    words = [stemmer.stem(w.lower()) for w in words if w.isalpha() and w.lower() not in stop_words]

    for word in set(words):
        for category in categories:
            feature_category_matrix[word][category] += 1

In [49]:
dft = pd.DataFrame(feature_category_matrix).fillna(0).T

In [50]:
dft

,trade,corn,grain,money-fx,interest,earn,wheat,crude,acq,ship
pressur,55.0,3.0,16.0,76.0,42.0,18.0,9.0,22.0,15.0,6.0
said,429.0,173.0,456.0,580.0,380.0,949.0,223.0,503.0,2143.0,256.0
disadvantag,4.0,0.0,1.0,2.0,0.0,2.0,1.0,0.0,1.0,1.0
larg,60.0,13.0,35.0,67.0,39.0,53.0,17.0,33.0,38.0,10.0
outcom,6.0,2.0,4.0,9.0,4.0,5.0,2.0,2.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...
genencor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
genecor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
genentech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
additivi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [36]:
dft['Total'] = dft.sum(axis=1)
dft_sorted = dft.sort_values(by='Total', ascending=False).drop('Total', axis=1)

In [37]:
dft_sorted.iloc[:50,:]

,trade,corn,grain,money-fx,interest,earn,wheat,crude,acq,ship
said,429.0,173.0,456.0,580.0,380.0,949.0,223.0,503.0,2143.0,256.0
lt,13.0,1.0,2.0,10.0,52.0,3643.0,1.0,173.0,2014.0,30.0
mln,148.0,105.0,228.0,182.0,103.0,2471.0,109.0,245.0,1066.0,50.0
dlr,232.0,48.0,137.0,180.0,98.0,1840.0,73.0,271.0,1338.0,44.0
year,280.0,109.0,231.0,164.0,81.0,1667.0,92.0,184.0,382.0,67.0
ct,1.0,8.0,9.0,2.0,1.0,2913.0,0.0,64.0,51.0,2.0
inc,4.0,3.0,12.0,18.0,22.0,1582.0,7.0,64.0,1281.0,13.0
vs,5.0,3.0,17.0,0.0,0.0,2931.0,11.0,0.0,1.0,0.0
pct,154.0,71.0,156.0,302.0,386.0,514.0,63.0,190.0,975.0,61.0
net,12.0,3.0,13.0,17.0,15.0,2638.0,3.0,30.0,106.0,2.0


In [55]:
lst=['compani','corp','market','sale','loss','bank','stock','rev','trade','report']

In [57]:
dft.loc[lst,:]

,trade,corn,grain,money-fx,interest,earn,wheat,crude,acq,ship
compani,49.0,10.0,28.0,30.0,13.0,935.0,14.0,210.0,1297.0,51.0
corp,20.0,6.0,24.0,15.0,25.0,1130.0,11.0,109.0,1075.0,23.0
market,200.0,68.0,148.0,423.0,260.0,180.0,74.0,130.0,320.0,24.0
sale,42.0,39.0,88.0,44.0,19.0,895.0,49.0,58.0,508.0,16.0
loss,17.0,8.0,23.0,22.0,11.0,1320.0,12.0,31.0,72.0,14.0
bank,94.0,1.0,9.0,457.0,356.0,248.0,3.0,39.0,299.0,13.0
stock,17.0,20.0,54.0,50.0,39.0,504.0,30.0,59.0,691.0,7.0
rev,0.0,0.0,0.0,0.0,0.0,1451.0,0.0,0.0,9.0,0.0
trade,451.0,60.0,134.0,237.0,108.0,94.0,61.0,68.0,161.0,42.0
report,98.0,78.0,152.0,85.0,28.0,412.0,69.0,171.0,190.0,87.0


### checking how many features of our reuters and research paper's reuters are same

In [38]:
l1=['plan', 'designer', 'kits', 'home', 'design', 'projects', 'sales', 'terms', 'disclosed', 'bank']

In [39]:
l2=list(dft_sorted.index)

In [40]:
common_words = [word for word in l1 if word in l2]

In [41]:
common_words

['plan', 'home', 'design', 'bank']

# implementing the cmfs algo from research paper

### code by abbas giving 0s and 1s only which is not required

In [51]:
# from collections import defaultdict
# from typing import List, Tuple, Dict, Any

# def feature_selection_algorithm(V: List[List[int]], k: int) -> List[int]:
#     # Step 1: obtain the number of categories (number of columns in V)
#     num_categories = len(V[0])

#     # Step 2: obtain the size of the feature vector space (number of rows in V)
#     num_features = len(V)

#     # Containers to store frequency and significance values
#     tf_ci = [0] * num_categories
#     tf_tk = [0] * num_features
#     tf_tk_ci = [[0] * num_categories for _ in range(num_features)]
#     CMFS_tk = [0] * num_features

#     # Step 3-5: Calculate tf(t, ci) for each category ci
#     for ci in range(num_categories):
#         tf_ci[ci] = sum(V[t][ci] for t in range(num_features))

#     # Step 6-12: Calculate CMFS(tk, ci) for each feature tk
#     for tk in range(num_features):
#         # Step 7: Calculate tf(tk) - frequency of feature tk in the entire training set
#         tf_tk[tk] = sum(V[tk])

#         for ci in range(num_categories):
#             # Step 9: Calculate tf(tk, ci) - frequency of feature tk in category ci
#             tf_tk_ci[tk][ci] = V[tk][ci]

#             # Step 10: Calculate CMFS(tk, ci)
#             CMFS_tk[tk] += tf_tk_ci[tk][ci] / tf_ci[ci] - tf_tk[tk] / num_features

#     # Step 14: Rank features based on CMFS
#     ranked_features = sorted(enumerate(CMFS_tk), key=lambda x: x[1], reverse=True)

#     # Step 15: Select top k features
#     selected_features = [feature for feature, _ in ranked_features[:k]]

#     return selected_features

# # Example usage:
# # V is a 2D list representing the term-to-category feature-appearance matrix
# # Each row corresponds to a feature, and each column corresponds to a category
# V = [
#     [1, 2, 0],
#     [0, 1, 1],
#     [2, 0, 1],
#     [1, 1, 0],
# ]

# # k = 2   Number of selected features

# selected_features = feature_selection_algorithm(dft.values.tolist(), k=10)
# print("Selected Features:", selected_features)


Selected Features: [12435, 1459, 1462, 3316, 3684, 3686, 3690, 3698, 3757, 3758]


In [52]:
# dft_sorted.iloc[selected_features,:]

,trade,corn,grain,money-fx,interest,earn,wheat,crude,acq,ship
aipn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
illinoi,4.0,7.0,8.0,0.0,2.0,8.0,0.0,1.0,14.0,4.0
chines,7.0,6.0,12.0,1.0,0.0,2.0,5.0,3.0,2.0,9.0
bow,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,3.0
dinner,2.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,3.0,0.0
unfavor,2.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0
mousavi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0
lifestyl,1.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,2.0,0.0
zoet,1.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,2.0,1.0
underwritten,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,6.0,1.0


### code by osama giving some desired results but not all

In [53]:
import numpy as np

def efficient_feature_selection(matrix, k):
    category_sums = matrix.sum(axis=0)

    significance_matrix = matrix / category_sums

    max_significance = significance_matrix.max(axis=1)

    top_features_indices = np.argsort(max_significance)[-k:][::-1]

    return top_features_indices

top_k_features = efficient_feature_selection(dft.to_numpy(), k=10)

top_k_features


array([  45,  799, 1412,  630,  196,  796,    1, 1400,  423,  237])

In [54]:
dft_sorted.iloc[top_k_features,:]

,trade,corn,grain,money-fx,interest,earn,wheat,crude,acq,ship
increas,107.0,46.0,100.0,84.0,71.0,346.0,42.0,139.0,180.0,40.0
packag,18.0,1.0,10.0,18.0,11.0,9.0,2.0,7.0,36.0,3.0
survey,4.0,1.0,6.0,11.0,5.0,3.0,3.0,15.0,1.0,2.0
lend,6.0,0.0,1.0,30.0,81.0,13.0,1.0,14.0,11.0,0.0
help,81.0,15.0,43.0,115.0,60.0,35.0,15.0,70.0,41.0,22.0
progress,36.0,2.0,6.0,13.0,5.0,27.0,2.0,4.0,16.0,4.0
lt,13.0,1.0,2.0,10.0,52.0,3643.0,1.0,173.0,2014.0,30.0
challeng,10.0,2.0,3.0,2.0,0.0,7.0,0.0,6.0,18.0,4.0
declar,9.0,1.0,3.0,7.0,0.0,193.0,1.0,9.0,19.0,4.0
transact,5.0,5.0,9.0,45.0,16.0,26.0,4.0,8.0,299.0,2.0
